In this notebook I'll reconstruct the process and results of the DTCR algorithm presented in the "Learning Representations for Time Series Clustering".

In [6]:
# Imports
import torch
from torch.utils.data import DataLoader
from Utilities.DTCR import DTCRModel, DTCRConfig
from Utilities.UCRParser import read_dataset

In [7]:
testing_dataset = "Symbols"
train_ds, test_ds = read_dataset(testing_dataset)

config = DTCRConfig()
config.class_num = train_ds.number_of_labels
config.input_size = train_ds[0][0].shape[1]
config.num_steps = train_ds[0][0].shape[0]
config.model_name = testing_dataset
config.batch_size = len(train_ds)

config.learning_rate = 5e-3
config.checkpoint_interval = 15
config.hidden_size = [50, 30, 30]
config.dilations = [1, 4, 16]
config.coefficient_lambda = 1

train_dl = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True, drop_last=True)
test_dl = DataLoader(test_ds, batch_size=len(test_ds), shuffle=True)

Loading the Two_Patterns dataset...
The dataset Two_Patterns was loaded.


In [8]:
dtcr_model = DTCRModel(config)
dtcr_model

DTCRModel(
  (encoder): BidirectionalDRNN(
    (_regular_drnn): DRNN(
      (_layers): Sequential(
        (0): GRU(1, 50, batch_first=True)
        (1): GRU(50, 30, batch_first=True)
        (2): GRU(30, 30, batch_first=True)
      )
    )
    (_backwards_drnn): DRNN(
      (_layers): Sequential(
        (0): GRU(1, 50, batch_first=True)
        (1): GRU(50, 30, batch_first=True)
        (2): GRU(30, 30, batch_first=True)
      )
    )
  )
  (decoder): DTCRDecoder(
    (_rnn): GRU(220, 220, batch_first=True)
    (_linear): Linear(in_features=220, out_features=1, bias=True)
  )
  (classifier): Sequential(
    (0): Linear(in_features=220, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)

In [9]:
# if you want to continue the training load the model here from the checkpoint
# dtcr_model = torch.load("Checkpoints/ECGFiveDays_900")
recons_criterion = config.decoding_criterion()
classify_criterion = config.classifier_criterion()
optimizer = config.optimizer(dtcr_model.parameters(),
                             eps=config.learning_rate)

In [10]:
epochs = 1000
for epoch in range(dtcr_model._training_iteration, epochs):
    print("Starting epoch {}:".format(epoch + 1))
    dtcr_model.train_step(train_dl, test_dl, recons_criterion, classify_criterion,
                          optimizer)

Starting epoch 1:
[1|1] loss: 3978.9307, classify: 0.69324,recons: 0.99303, clustering: 3977.24438
Starting epoch 2:
[2|1] loss: 3534.0952, classify: 0.69321,recons: 0.99227, clustering: 3532.40967
Starting epoch 3:
[3|1] loss: 3161.3992, classify: 0.69326,recons: 0.99210, clustering: 3159.71387
Starting epoch 4:
[4|1] loss: 2846.3457, classify: 0.69320,recons: 0.99218, clustering: 2844.66040
Starting epoch 5:
[5|1] loss: 2580.3909, classify: 0.69318,recons: 0.99228, clustering: 2578.70532
Starting epoch 6:
[6|1] loss: 2354.8247, classify: 0.69316,recons: 0.99232, clustering: 2353.13916
Starting epoch 7:


KeyboardInterrupt: 